In [7]:
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torchsummary import summary
from sklearn.metrics import explained_variance_score
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = np.load('valid_100K_disc0.99_cleaned.npz')

In [5]:
x_train = data['observations']
p_train = data['actions']
v_train = data['rewards']
x_train.shape, p_train.shape, v_train.shape

((95623, 11, 11, 18), (95623,), (95623,))

In [8]:
class LinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        input_size = 11*11*18
        self.fc_p = nn.Linear(input_size, 6)
        self.fc_v = nn.Linear(input_size, 1)
    
    def forward(self, x):
        p = F.softmax(self.fc_p(x))
        v = F.tanh(self.fc_v(x))

In [19]:
model = LinearModel()
optimizer = optim.Adam()
summary(model, input_size=(11*11*18,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 6]          13,074
            Linear-2                    [-1, 1]           2,179
Total params: 15,253
Trainable params: 15,253
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.00
Params size (MB): 0.06
Estimated Total Size (MB): 0.07
----------------------------------------------------------------


In [13]:
summary?

Signature: summary(model, input_size, batch_size=-1, device='cuda')
Docstring: <no docstring>
File:      ~/Programming/pommerman-baselines/.venv/lib/python3.8/site-packages/torchsummary/torchsummary.py
Type:      function
